In [111]:
using MAGEMin_C
using DataFrames
using CSV
using Optim
using Tables

## Calculate for fiducial Enst composition & metal depleted

In [ ]:
db = "ig"
sys_in  = "wt"

# pressure in kbar
P = 50.
# temperature 
T = LinRange(2200, 1000, 20)
buffer = "qfm"
B= -5.

database = Initialize_MAGEMin(db, verbose=false, buffer=buffer)

dfCompositions = CSV.read("thermoInput.csv", DataFrame)
dfCompositionsLSi = CSV.read("thermoInputLSi.csv", DataFrame)

# set oxide order based off of data frame, catching the meta data at beginning
Xoxides = names(dfCompositions[1,2:end])

cOutputs = []
for row in eachrow(dfCompositions)
    X = Array(row[2:end])
    tOutputs = []
    for t in T
        tout = single_point_minimization(P, t, database, X=X, Xoxides=Xoxides, sys_in=sys_in, B=B)
        push!(tOutputs, tout)
    end
    push!(cOutputs, (row, tOutputs))
end

In [406]:
cOutputsLSi = []
for row in eachrow(dfCompositionsLSi)
    X = Array(row[2:end])
    tOutputs = []
    for t in T
        tout = single_point_minimization(P, t, database, X=X, Xoxides=Xoxides, sys_in=sys_in, B=B)
        push!(tOutputs, tout)
    end
    push!(cOutputsLSi, (row, tOutputs))
end

In [407]:
# Regular Si
phs = [ph.ph for ph in [cOutputs[i][2][end] for i in eachindex(cOutputs)]]
uniquePhs = unique(mapreduce(permutedims, hcat, phs))
colnames = vcat(["coreSi", "mgsi", "P", "T"], uniquePhs)

dfSubSol = DataFrame([name => Float64[] for name = colnames])
for i in eachindex(cOutputs)
    coreSi = cOutputs[i][1][1]
    mgsi = cOutputs[i][1].MgO / cOutputs[i][1].SiO2 * ((28.084 + 15.999*2) / (24.305 + 15.9994))
    t = cOutputs[i][2][end].T_C
    p = cOutputs[i][2][end].P_kbar
    push!(dfSubSol, vcat([coreSi, mgsi, p, t], zeros(length(uniquePhs)) ))
    for (j, p) in enumerate(cOutputs[i][2][end].ph)
        if p != "qfm"
            dfSubSol[end, p] = cOutputs[i][2][end].ph_frac_wt[j]
        end
    end
end

CSV.write("subSol.csv", dfSubSol)

# Low Si
phs = [ph.ph for ph in [cOutputsLSi[i][2][end] for i in eachindex(cOutputsLSi)]]
uniquePhs = unique(mapreduce(permutedims, hcat, phs))
colnames = vcat(["coreSi", "mgsi", "P", "T"], uniquePhs)

dfSubSolLSi = DataFrame([name => Float64[] for name = colnames])
for i in eachindex(cOutputsLSi)
    coreSi = cOutputsLSi[i][1][1]
    mgsi = cOutputsLSi[i][1].MgO / cOutputsLSi[i][1].SiO2 * ((28.084 + 15.999*2) / (24.305 + 15.9994))
    t = cOutputsLSi[i][2][end].T_C
    p = cOutputsLSi[i][2][end].P_kbar
    push!(dfSubSolLSi, vcat([coreSi, mgsi, p, t], zeros(length(uniquePhs)) ))
    for (j, p) in enumerate(cOutputsLSi[i][2][end].ph)
        if p != "qfm"
            dfSubSolLSi[end, p] = cOutputsLSi[i][2][end].ph_frac_wt[j]
        end
    end
end

CSV.write("subSolLSi.csv", dfSubSolLSi)

"subSolLSi.csv"

In [404]:
cOutputs[end][2][7]

Pressure          : 50.0      [kbar]
Temperature       : 1821.0526    [Celsius]
     Stable phase | Fraction (mol fraction) 
               ol   0.25336 
              liq   0.1361 
              cpx   0.57659 
              opx   0.03395 
              qfm   0.0 
     Stable phase | Fraction (wt fraction) 
               ol   0.23298 
              liq   0.14157 
              cpx   0.59112 
              opx   0.03433 
              qfm   0.0 
     Stable phase | Fraction (vol fraction) 
               ol   0.23346 
              liq   0.15347 
              cpx   0.57942 
              opx   0.03365 
              qfm   0.0 
Gibbs free energy : -898.811335  (221 iterations; 119.18 ms)
Oxygen fugacity          : -5.9631413134683005
Delta QFM                : -5.00010094296507


Si activity calculator

In [ ]:
db = "ig"
sys_in  = "wt"

# pressure in kbar
# temperature 
buffer = "qfm"
B= -5.

database = Initialize_MAGEMin(db, verbose=false, buffer=buffer)

dfCompositions = CSV.read("thermoInput.csv", DataFrame)
dfCompositionsLSi = CSV.read("thermoInputLSi.csv", DataFrame)

# set oxide order based off of data frame, catching the meta data at beginning
Xoxides = names(dfCompositions[1,2:end])

cOutputs = []
# for row in eachrow(dfCompositions)
i = 70
X = Array(dfCompositions[i,2:end])
t = 1500.
function fun(t, p)
    
    tout = single_point_minimization(p, t[1], database, X=X, Xoxides=Xoxides, sys_in=sys_in, B=B)

    mf = 0
    try
        iLiq = findall(x->x=="liq", tout.ph)[1]
        fMelt = tout.ph_frac_wt[iLiq]
        mf = (0.05 - fMelt)^2
    catch e
        mf = 2
    end
    mf
end

x0 = [1900.]
P = LinRange(50, 10, 20)

tps = zeros((length(P), 3))
for (i, p) in enumerate(P)
    print(p,'\r')
    res = optimize((x -> fun(x, p)), x0)

    tout = single_point_minimization(p, Optim.minimizer(res)[1], database, X=X, Xoxides=Xoxides, sys_in=sys_in, B=B)
    iLiq = findall(x->x=="liq", tout.ph)[1]
    fMelt = tout.ph_frac_wt[iLiq]

    tps[i,:] = [p, Optim.minimizer(res)[1], fMelt]

    x0[1] = Optim.minimizer(res)[1]
    
end

# push!(cOutputs, (dfCompositions[i,:], tout))

# end

In [112]:
CSV.write("tp.csv", Tables.table(tps), writeheader=false)

"tp.csv"

In [109]:
tps

10×3 Matrix{Float64}:
 50.0     1806.39  0.0499963
 45.5556  1781.04  0.050033
 41.1111  1748.8   0.0499704
 36.6667  1707.81  0.0498272
 32.2222  1656.31  0.050012
 27.7778  1592.02  0.0499393
 23.3333  1512.92  0.0499574
 18.8889  1414.66  0.0499527
 14.4444  1286.87  0.0499579
 10.0     1141.28  0.0499565